# E-tivity 2: Data Preperation
## Collin Wyer (14103001)


### Task: 
 - encoded all categorical attributes as numerical,
 - filled in all missing values
 - applied a scaler to rescale features
 - treated outliers appropriately
 - created at least one new feature from the existing ones

# Step 1

In [49]:
# Import Python Modules
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.impute import MissingIndicator
import matplotlib.pyplot as plt
%matplotlib inline

# Read file

df= pd.read_csv("bank.csv")

In [3]:
# Select sample to view

df.sample(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
1010,40.0,technician,married,tertiary,no,11766,yes,no,cellular,13,apr,218,2,242,9,failure,yes
625,29.0,admin.,single,secondary,no,2768,yes,no,cellular,29,jan,69,1,241,4,failure,no
894,27.0,services,married,primary,no,91,yes,no,NaN,8,may,168,2,-1,0,NaN,no
391,42.0,management,married,primary,no,-417,yes,no,cellular,5,feb,81,3,273,3,failure,no
1683,57.0,retired,married,primary,no,9367,no,no,cellular,14,may,700,4,92,7,success,yes
276,33.0,management,single,tertiary,no,191,no,yes,cellular,3,feb,678,4,209,1,failure,no
1165,55.0,technician,married,NaN,no,240,yes,no,cellular,2,aug,108,3,277,1,success,yes
1765,54.0,admin.,married,secondary,no,66,yes,no,cellular,11,nov,167,1,-1,0,NaN,yes
1676,35.0,management,married,tertiary,no,8000,no,no,cellular,13,may,331,1,90,3,success,yes
1322,36.0,technician,divorced,secondary,no,1451,yes,no,cellular,23,jul,192,4,93,17,other,yes


We can already see there are missing values present and/or values that will require 

In [11]:
# Determine the number of nulls in the dataset

df.isna().sum()

age            12
job            10
marital         0
education     104
default         0
balance         0
housing         0
loan            0
contact       191
day             0
month           0
duration        0
campaign        0
pdays           0
previous        0
poutcome      454
subscribed      0
dtype: int64

In [14]:
len(df)

2000

No attribute nulls are > 25% so all can be kept.  

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 17 columns):
age           1988 non-null float64
job           1990 non-null object
marital       2000 non-null object
education     1896 non-null object
default       2000 non-null object
balance       2000 non-null int64
housing       2000 non-null object
loan          2000 non-null object
contact       1809 non-null object
day           2000 non-null int64
month         2000 non-null object
duration      2000 non-null int64
campaign      2000 non-null int64
pdays         2000 non-null int64
previous      2000 non-null int64
poutcome      1546 non-null object
subscribed    2000 non-null object
dtypes: float64(1), int64(6), object(10)
memory usage: 265.8+ KB


In [51]:
# Create a copy before transforming

df_orginial= df

In [52]:
#Drop columns with useless information

df.drop('contact', axis=1, inplace=True)
df.sample(5)

,age,job,marital,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,poutcome,subscribed
1113,30.0,admin.,single,secondary,no,915,yes,no,27,apr,370,1,-1,0,NaN,yes
459,35.0,management,single,secondary,no,6809,yes,no,6,feb,139,8,273,7,failure,no
821,38.0,management,married,tertiary,no,66,yes,no,8,may,124,1,-1,0,NaN,no
1605,44.0,admin.,divorced,tertiary,no,558,yes,no,31,mar,262,2,-1,0,NaN,yes
384,30.0,blue-collar,single,primary,no,105,yes,no,5,feb,131,1,265,1,failure,no


 ### Encoded all categorical attributes as numerical


In [53]:
# Mappers

# marital
martial_scale_mapper= {"single": 0, "married": 1, "divorced": 2}
df['marital']= df['marital'].replace(martial_scale_mapper)

# education
edu_scale_mapper= {"primary": 0, "secondary": 1, "tertiary": 2}
df['education']= df['education'].replace(edu_scale_mapper)

# month
month_scale_mapper= {"jan":0, "feb":1, "mar":2, "apr":3, "may":4, 
               "jun":5, "jul":6, "aug":7, "sep":8, "oct":9, 
               "nov":10, "dec":11}
df['month']= df['month'].replace(month_scale_mapper)

# poutcome
poutcome_scale_mapper = {"failure": 1, "success": 2, "other": 3}
df['poutcome'] = df['poutcome'].replace(poutcome_scale_mapper)

df.sample(5)

,age,job,marital,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,poutcome,subscribed
708,54.0,admin.,1,1.0,no,263,no,yes,29,0,283,4,195,7,1.0,no
1490,45.0,technician,0,2.0,no,2033,no,no,15,5,243,5,285,2,2.0,yes
1721,84.0,retired,2,0.0,no,639,no,no,18,4,353,3,-1,0,NaN,yes
1767,29.0,management,0,2.0,no,765,no,no,16,10,238,1,-1,0,NaN,yes
331,33.0,blue-collar,1,1.0,no,-241,yes,yes,4,1,117,1,253,1,1.0,no


### One-hot encode for all other categorical

In [54]:
# one-hot encode for Nans

df= pd.get_dummies(df, columns=['job', 'housing', 'default', 'poutcome'], 
                  dummy_na=True, drop_first=True)
df.sample(5)

,age,marital,education,balance,loan,day,month,duration,campaign,pdays,...,job_technician,job_unemployed,job_nan,housing_yes,housing_nan,default_yes,default_nan,poutcome_2.0,poutcome_3.0,poutcome_nan
1605,44.0,2,2.0,558,no,31,2,262,2,-1,...,0,0,0,1,0,0,0,0,0,1
284,59.0,1,2.0,941,yes,4,1,231,1,210,...,0,0,0,0,0,0,0,0,0,0
593,50.0,0,2.0,567,no,29,0,50,1,258,...,0,0,0,1,0,0,0,0,0,0
1527,38.0,2,2.0,2946,no,16,5,605,3,-1,...,0,0,0,0,0,0,0,0,0,1
1677,68.0,1,1.0,1146,no,13,4,356,1,71,...,0,0,0,0,0,0,0,1,0,0


In [55]:
# one-hot encode all other categorical

df=pd.get_dummies(df, drop_first=True)
df.sample(5)

,age,marital,education,balance,day,month,duration,campaign,pdays,previous,...,job_nan,housing_yes,housing_nan,default_yes,default_nan,poutcome_2.0,poutcome_3.0,poutcome_nan,loan_yes,subscribed_yes
1523,27.0,0,NaN,861,14,5,212,4,-1,0,...,0,0,0,0,0,0,0,1,0,1
1100,34.0,1,2.0,589,19,3,125,2,-1,0,...,0,0,0,0,0,0,0,1,0,1
1607,37.0,1,2.0,2283,6,4,243,2,98,2,...,0,0,0,0,0,0,0,0,0,1
50,43.0,2,1.0,266,20,3,79,1,152,1,...,0,1,0,0,0,0,0,0,1,0
585,36.0,1,1.0,498,29,0,151,1,253,1,...,0,1,0,0,0,0,0,0,0,0


In [56]:
df.isna().sum()


age                   12
marital                0
education            104
balance                0
day                    0
month                  0
duration               0
campaign               0
pdays                  0
previous               0
job_blue-collar        0
job_entrepreneur       0
job_housemaid          0
job_management         0
job_retired            0
job_self-employed      0
job_services           0
job_student            0
job_technician         0
job_unemployed         0
job_nan                0
housing_yes            0
housing_nan            0
default_yes            0
default_nan            0
poutcome_2.0           0
poutcome_3.0           0
poutcome_nan           0
loan_yes               0
subscribed_yes         0
dtype: int64

## Save the dataset

In [ ]:
df.to_csv('bank_prepared.csv')